In [1]:
from aiida import load_profile
from aiida.orm import Int
from aiida_workgraph import task, WorkGraph

In [2]:
from pickle import loads

In [3]:
@task.calcfunction(outputs=[{"name": "a"}, {"name": "b"}, {"name": "c"}])
def add_x_and_y(x, y):
    z = x + y
    a = x * 1.0
    b = y * 1.0
    return {"a": a, "b": b, "c": z}

In [4]:
@task.calcfunction()
def add_x_and_y_and_z(l, m, n):
    w = l + m + n
    return w

In [5]:
load_profile()

Profile<uuid='7bb8761123324468bb98821cbb757251' name='presto'>

In [6]:
wg = WorkGraph("my_workflow")
result = wg.add_task(add_x_and_y, name="add_x_and_y", x=1, y=2)
result_2 = wg.add_task(
    add_x_and_y_and_z, 
    name="add_x_and_y_and_z", 
    l=result.outputs.c, 
    m=result.outputs.a, 
    n=result.outputs.b,
)

In [7]:
work_graph_dict = wg.to_dict()

In [8]:
edges_label_lst = []
for link_dict in work_graph_dict["links"]:
    if link_dict['from_socket'] == "result":
        edges_label_lst.append({'target': link_dict['to_node'], 'targetHandle': link_dict['to_socket'], 'source': link_dict['from_node'], 'sourceHandle': None})
    else:
        edges_label_lst.append({'target': link_dict['to_node'], 'targetHandle': link_dict['to_socket'], 'source': link_dict['from_node'], 'sourceHandle': link_dict['from_socket']})

edges_label_lst

[{'target': 'add_x_and_y_and_z',
  'targetHandle': 'l',
  'source': 'add_x_and_y',
  'sourceHandle': 'c'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'm',
  'source': 'add_x_and_y',
  'sourceHandle': 'a'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'n',
  'source': 'add_x_and_y',
  'sourceHandle': 'b'}]

In [9]:
kwargs_dict, function_dict = {}, {}
for task_name, task_dict in work_graph_dict["tasks"].items():
    input_variables = [
        input_parameter 
        for input_parameter in task_dict['inputs'].keys() 
        if not input_parameter.startswith("metadata") and not input_parameter.startswith("_wait")
    ]
    input_kwargs = {
        input_parameter: task_dict['inputs'][input_parameter]['property']["value"].value if isinstance(task_dict['inputs'][input_parameter]['property']["value"], dict) else task_dict['inputs'][input_parameter]['property']["value"] 
        for input_parameter in input_variables
    }
    kwargs_dict[task_name] = input_kwargs
    function_dict[task_name] = task_dict['executor']['callable'].process_class._func
kwargs_dict, function_dict

({'add_x_and_y': {'x': 1, 'y': 2},
  'add_x_and_y_and_z': {'l': None, 'm': None, 'n': None}},
 {'add_x_and_y': <function __main__.add_x_and_y(x, y)>,
  'add_x_and_y_and_z': <function __main__.add_x_and_y_and_z(l, m, n)>})

In [10]:
nodes_dict, mapping_dict = {}, {}
for i, [k, v] in enumerate(function_dict.items()):
    nodes_dict[i] = v
    mapping_dict[k] = i

nodes_dict, mapping_dict 

({0: <function __main__.add_x_and_y(x, y)>,
  1: <function __main__.add_x_and_y_and_z(l, m, n)>},
 {'add_x_and_y': 0, 'add_x_and_y_and_z': 1})

In [11]:
value_dict = {}
for func_name, val_dict in kwargs_dict.items():
    for k, v in val_dict.items():
        if v is not None:
            if func_name not in value_dict.keys():
                value_dict[func_name] = {}
            value_dict[func_name][k] = v

In [12]:
i = len(nodes_dict)
for val_dict in value_dict.values():
    for k, v in val_dict.items():
        nodes_dict[i] = v
        mapping_dict[v] = i
        i += 1

nodes_dict, mapping_dict

({0: <function __main__.add_x_and_y(x, y)>,
  1: <function __main__.add_x_and_y_and_z(l, m, n)>,
  2: 1,
  3: 2},
 {'add_x_and_y': 0, 'add_x_and_y_and_z': 1, 1: 2, 2: 3})

In [13]:
for func_name, val_dict in kwargs_dict.items():
    for k, v in val_dict.items():
        if v is not None:
            edges_label_lst.append({'target': func_name, 'targetHandle': k, 'source': v, 'sourceHandle': None})
edges_label_lst

[{'target': 'add_x_and_y_and_z',
  'targetHandle': 'l',
  'source': 'add_x_and_y',
  'sourceHandle': 'c'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'm',
  'source': 'add_x_and_y',
  'sourceHandle': 'a'},
 {'target': 'add_x_and_y_and_z',
  'targetHandle': 'n',
  'source': 'add_x_and_y',
  'sourceHandle': 'b'},
 {'target': 'add_x_and_y',
  'targetHandle': 'x',
  'source': 1,
  'sourceHandle': None},
 {'target': 'add_x_and_y',
  'targetHandle': 'y',
  'source': 2,
  'sourceHandle': None}]

In [14]:
edges_lst = []
for edge in edges_label_lst:
    edges_lst.append({'target': mapping_dict[edge['target']], 'targetHandle': edge['targetHandle'], 'source': mapping_dict[edge['source']], 'sourceHandle': edge['sourceHandle']})

edges_lst

[{'target': 1, 'targetHandle': 'l', 'source': 0, 'sourceHandle': 'c'},
 {'target': 1, 'targetHandle': 'm', 'source': 0, 'sourceHandle': 'a'},
 {'target': 1, 'targetHandle': 'n', 'source': 0, 'sourceHandle': 'b'},
 {'target': 0, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
 {'target': 0, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None}]

In [15]:
nodes_dict

{0: <function __main__.add_x_and_y(x, y)>,
 1: <function __main__.add_x_and_y_and_z(l, m, n)>,
 2: 1,
 3: 2}

In [16]:
load_profile()

Profile<uuid='7bb8761123324468bb98821cbb757251' name='presto'>

In [17]:
wg.run()

01/17/2025 06:18:06 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|continue_workgraph]: Continue workgraph.
01/17/2025 06:18:06 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y
01/17/2025 06:18:06 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|run_tasks]: Run task: add_x_and_y, type: CALCFUNCTION


------------------------------------------------------------
kwargs:  {'x': 1, 'y': 2}


01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|update_task_state]: Task: add_x_and_y finished.
01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|continue_workgraph]: Continue workgraph.
01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y_and_z
01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|run_tasks]: Run task: add_x_and_y_and_z, type: CALCFUNCTION


------------------------------------------------------------
kwargs:  {'l': <Int: uuid: 1d416b15-849e-487c-8cdb-b12db8738b39 (pk: 271) value: 3>, 'm': <Float: uuid: d068d910-dccc-43c5-b88d-115c76836bee (pk: 269) value: 1.0>, 'n': <Float: uuid: 2735d182-e653-4641-b13f-eff9eba01406 (pk: 270) value: 2.0>}


01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|update_task_state]: Task: add_x_and_y_and_z finished.
01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|continue_workgraph]: Continue workgraph.
01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
01/17/2025 06:18:07 PM <120623> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [265|WorkGraphEngine|finalize]: Finalize workgraph.


{'execution_count': <Int: uuid: acbe352e-6c52-44eb-a1a2-99d249f99867 (pk: 274) value: 1>}

In [18]:
result_2.outputs.result

SocketAny(name='result', value=uuid: a0410fcf-4ef0-47e2-83d4-6cfc3521c593 (pk: 273) value: 6.0)